In [119]:
import requests
import pandas as pd
import json

In [2]:
# Esta funcion imprime las 10 monedas volatiles con mas capitalizacion y luego las 10 monedas estables.
def get_top_coins():
    stable = []
    volatil = []

    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 100,
        "page": 1
    }

    response = requests.get(url, params=params)
    data = response.json()

    for coin in data:
        if coin['current_price'] > 1 or 0.99 > coin['current_price'] and len(volatil) < 10:
            volatil.append(coin['id'])
        elif 1.1 > coin['current_price'] > 0.97 and len(stable) < 10:
            stable.append(coin['id'])
        else:
            break
        
    return stable,volatil

stable, volatil = get_top_coins()
print(stable)
print(volatil)

['tether', 'usd-coin']
['bitcoin', 'ethereum', 'binancecoin', 'ripple', 'staked-ether', 'cardano', 'dogecoin', 'solana', 'tron', 'polkadot']


In [185]:
# Utiliza una lista de nombres de criptomonedas y a traves de la api de coingecko extrae la informacion de cada una de ellas para luego de un ETL guardar 
# la informacion en un archivo csv.
def get_historical_data(coinnamelist,days):
    for coin in coinnamelist:
        url = f"https://api.coingecko.com/api/v3/coins/{coin}/market_chart"
        params = {
            'vs_currency': 'usd',
            'days': days,
            'interval': 'daily'
        }
    
        response = requests.get(url, params=params)
    
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data)
            df['market_caps'] = df['market_caps'].apply(lambda x: int(x[1]))
            df['total_volumes'] = df['total_volumes'].apply(lambda x: int(x[1]))
            df['Date'] = df['prices'].apply(lambda x: pd.to_datetime(x[0], unit='ms'))
            df['prices'] = df['prices'].apply(lambda x: int(x[1]))
            df.rename(columns={'prices': 'Price_usd','total_volumes':'Volume','market_caps':'Market_cap'}, inplace=True)
            df = df.groupby(df["Date"].dt.to_period("M")).tail(1)
            df.to_csv(f'./Data/{coin}.csv',index=False)
        else:
            print(f"Error al obtener los datos de {coin}:", response.status_code)


In [28]:
get_historical_data(volatil)

In [147]:
url = 'https://api.coingecko.com/api/v3/global'
response = requests.get(url)
if response.status_code == 200:
    data = response.json()

In [148]:
data

{'data': {'active_cryptocurrencies': 10156,
  'upcoming_icos': 0,
  'ongoing_icos': 49,
  'ended_icos': 3376,
  'markets': 832,
  'total_market_cap': {'btc': 41837846.94374321,
   'eth': 665639519.9273119,
   'ltc': 16815415391.979414,
   'bch': 5821240400.536904,
   'bnb': 5101706192.826799,
   'eos': 1882626029844.5295,
   'xrp': 2089763957938.195,
   'xlm': 9031000954999.605,
   'link': 177438831271.22357,
   'dot': 247924837738.57086,
   'yfi': 195961445.20286462,
   'usd': 1088649910932.4526,
   'aed': 3998649225601.777,
   'ars': 381009070642863.1,
   'aud': 1691157760888.4636,
   'bdt': 118933971817904.67,
   'bhd': 410407952622.60284,
   'bmd': 1088649910932.4526,
   'brl': 5376079855157.736,
   'cad': 1473874125165.453,
   'chf': 957449089616.6056,
   'clp': 946635530051312.2,
   'cny': 7822657232491.751,
   'czk': 24127857929636.91,
   'dkk': 7470315688818.485,
   'eur': 1002313441096.0427,
   'gbp': 853555962666.5895,
   'hkd': 8531479366799.694,
   'huf': 383150322000227.3,

In [149]:
data = data['data']
df_global = pd.DataFrame(data)
df_global.head()

,active_cryptocurrencies,upcoming_icos,ongoing_icos,ended_icos,markets,total_market_cap,total_volume,market_cap_percentage,market_cap_change_percentage_24h_usd,updated_at
btc,10156,0,49,3376,832,4.183785e+07,1.651133e+06,46.525856,-0.209935,1692768339
eth,10156,0,49,3376,832,6.656395e+08,2.626951e+07,18.064198,-0.209935,1692768339
ltc,10156,0,49,3376,832,1.681542e+10,6.636214e+08,NaN,-0.209935,1692768339
bch,10156,0,49,3376,832,5.821240e+09,2.297356e+08,NaN,-0.209935,1692768339
bnb,10156,0,49,3376,832,5.101706e+09,2.013392e+08,3.015820,-0.209935,1692768339


In [150]:
df_global.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67 entries, btc to sol
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   active_cryptocurrencies               67 non-null     int64  
 1   upcoming_icos                         67 non-null     int64  
 2   ongoing_icos                          67 non-null     int64  
 3   ended_icos                            67 non-null     int64  
 4   markets                               67 non-null     int64  
 5   total_market_cap                      61 non-null     float64
 6   total_volume                          61 non-null     float64
 7   market_cap_percentage                 10 non-null     float64
 8   market_cap_change_percentage_24h_usd  67 non-null     float64
 9   updated_at                            67 non-null     int64  
dtypes: float64(4), int64(6)
memory usage: 5.8+ KB


In [151]:
df_global['market_cap_percentage'].sum()

83.78180188576859

In [153]:
df_global.head()

,active_cryptocurrencies,upcoming_icos,ongoing_icos,ended_icos,markets,total_market_cap,total_volume,market_cap_percentage,market_cap_change_percentage_24h_usd,updated_at
btc,10156,0,49,3376,832,4.183785e+07,1.651133e+06,46.525856,-0.209935,2023-08-23 05:25:39
eth,10156,0,49,3376,832,6.656395e+08,2.626951e+07,18.064198,-0.209935,2023-08-23 05:25:39
ltc,10156,0,49,3376,832,1.681542e+10,6.636214e+08,NaN,-0.209935,2023-08-23 05:25:39
bch,10156,0,49,3376,832,5.821240e+09,2.297356e+08,NaN,-0.209935,2023-08-23 05:25:39
bnb,10156,0,49,3376,832,5.101706e+09,2.013392e+08,3.015820,-0.209935,2023-08-23 05:25:39


In [170]:
df_global.drop(delete_columns,axis=1,inplace=True)

In [171]:
df_global.head()

,total_market_cap,total_volume,market_cap_percentage
btc,4.183785e+07,1.651133e+06,46.525856
eth,6.656395e+08,2.626951e+07,18.064198
ltc,1.681542e+10,6.636214e+08,NaN
bch,5.821240e+09,2.297356e+08,NaN
bnb,5.101706e+09,2.013392e+08,3.015820


In [184]:
df_global = df_global.sort_values(by='market_cap_percentage', ascending=False)
df_global.head()

,total_market_cap,total_volume,market_cap_percentage
btc,4.183785e+07,1.651133e+06,46.525856
eth,6.656395e+08,2.626951e+07,18.064198
usdt,0.000000e+00,0.000000e+00,7.602511
bnb,5.101706e+09,2.013392e+08,3.015820
xrp,2.089764e+12,8.247267e+10,2.530325


In [180]:
df_global = df_global.fillna(0)

In [181]:
df_global[['total_market_cap','total_volume']].astype(int)

,total_market_cap,total_volume
btc,41837846,1651133
eth,665639519,26269505
usdt,0,0
bnb,-2147483648,201339155
xrp,-2147483648,-2147483648
...,...,...
xag,-2147483648,1821612245
ltc,-2147483648,663621425
bch,-2147483648,229735618
xau,572085528,22577391


***

In [198]:
# Esta funcion imprime las 10 monedas volatiles con mas capitalizacion y luego las 10 monedas estables.
def all_coins():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 300,
        "page": 1
    }

    response = requests.get(url, params=params)
    data = response.json()
        
    return data

In [199]:
df_global = all_coins()

In [200]:
df = pd.DataFrame(df_global)
df

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,26065.000000,506945600235,1,5.469101e+11,1.471501e+10,26155.000000,...,2.100000e+07,2.100000e+07,69045.000000,-62.27536,2021-11-10T14:24:11.849Z,67.810000,38312.14205,2013-07-06T00:00:00.000Z,None,2023-08-23T06:23:55.953Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1640.440000,197050578417,2,1.970506e+11,1.097422e+10,1666.860000,...,1.202143e+08,NaN,4878.260000,-66.39997,2021-11-10T14:24:19.604Z,0.432979,378462.82461,2015-10-20T00:00:00.000Z,"{'times': 83.16680152029345, 'currency': 'btc'...",2023-08-23T06:23:54.088Z
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.999161,82783800019,3,8.278380e+10,2.280178e+10,1.004000,...,8.285333e+10,NaN,1.320000,-24.48308,2018-07-24T00:00:00.000Z,0.572521,74.51950,2015-03-02T00:00:00.000Z,None,2023-08-23T06:20:00.886Z
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,214.170000,32905876804,4,4.277486e+10,8.218415e+08,214.430000,...,1.538562e+08,2.000000e+08,686.310000,-68.83692,2021-05-10T07:24:17.097Z,0.039818,537033.78183,2017-10-19T00:00:00.000Z,None,2023-08-23T06:23:57.190Z
4,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,0.519723,27461553758,5,5.196931e+10,1.126231e+09,0.522798,...,9.998850e+10,1.000000e+11,3.400000,-84.70538,2018-01-07T00:00:00.000Z,0.002686,19249.96857,2014-05-22T00:00:00.000Z,None,2023-08-23T06:23:56.471Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,hxro,hxro,HXRO,https://assets.coingecko.com/coins/images/7805...,0.149903,91438158,246,1.498039e+08,3.871850e+03,0.152546,...,1.000000e+09,NaN,0.800981,-81.29751,2021-06-05T00:59:41.210Z,0.009147,1537.77206,2019-11-10T15:12:15.992Z,None,2023-08-23T06:23:07.812Z
246,lido-staked-sol,stsol,Lido Staked SOL,https://assets.coingecko.com/coins/images/1836...,23.170000,90286000,247,9.028589e+07,1.015067e+06,23.780000,...,3.900407e+06,3.900407e+06,263.240000,-91.19861,2021-11-07T01:28:34.051Z,8.650000,167.69984,2022-12-29T20:46:29.701Z,None,2023-08-23T06:23:08.380Z
247,nervos-network,ckb,Nervos Network,https://assets.coingecko.com/coins/images/9566...,0.002682,89281131,248,NaN,1.653119e+06,0.002690,...,NaN,NaN,0.043706,-93.87235,2021-03-31T17:08:59.155Z,0.001979,35.33488,2022-12-17T14:25:57.243Z,"{'times': -0.731814045904007, 'currency': 'usd...",2023-08-23T06:23:54.574Z
248,cartesi,ctsi,Cartesi,https://assets.coingecko.com/coins/images/1103...,0.119709,88988460,249,1.196814e+08,3.976349e+06,0.120713,...,1.000000e+09,1.000000e+09,1.740000,-93.14946,2021-05-09T00:43:41.306Z,0.026706,347.40846,2020-05-10T04:49:27.744Z,None,2023-08-23T06:23:51.865Z


In [202]:
df[df['id'] == 'bitcoin']['market_cap'][0]/df['market_cap'].sum()

0.4759962149362621